In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

In [2]:
train_joke_df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')

In [3]:
test_joke_df_nofactrating = pd.read_csv(r'data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

In [4]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.005, random_state=42)

In [5]:
rows, r_pos = np.unique(train_df.values[:,0], return_inverse=True)
cols, c_pos = np.unique(train_df.values[:,1], return_inverse=True)

In [6]:
interactions_sparse = sparse.csr_matrix((train_df.values[:,2], (r_pos, c_pos)) )

In [7]:
interactions_sparse

<24983x100 sparse matrix of type '<class 'numpy.float64'>'
	with 1441122 stored elements in Compressed Sparse Row format>

In [8]:
Pui = normalize(interactions_sparse, norm='l2', axis=1)


#Похожесть айтемов между собой
sim = Pui.T * Pui

In [9]:
n_neighbors = list(range(5, 31))

ratings_count = [[] for _ in range(len(n_neighbors))]
result = [[] for _ in range(len(n_neighbors))]
err =  [0 for _ in range(len(n_neighbors))]


for i, row in enumerate(tqdm(valid_df.values)):
    user = int(row[0])
    joke = int(row[1])
    
    user_id = int(row[0]) - 1
    joke_id = int(row[1]) - 1
    
    for nn_ind, nn in enumerate(n_neighbors):
        sim_jokes = [i for i in sim[joke_id].toarray().argsort()[0][-nn:]]
        ratings = []
    
        for j in sim_jokes:
            rating_df = train_df.loc[(train_df.UID == user) & (train_df.JID == j)].values
            if len(rating_df) > 0:
                rating = rating_df[0][2]
                ratings.append(rating)

        if len(ratings) == 0:
            err[nn_ind] += 1
            #print(nn, err[nn_ind])
            
        ratings_count[nn_ind].append(len(ratings))
        result[nn_ind].append(np.mean(ratings))

  0%|          | 0/7242 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
for r, n in zip(result, n_neighbors):
    valid_df[f'ratind_{n}'] = r

In [11]:
valid_df

,UID,JID,Rating,ratind_5,ratind_6,ratind_7,ratind_8,ratind_9,ratind_10,ratind_11,...,ratind_21,ratind_22,ratind_23,ratind_24,ratind_25,ratind_26,ratind_27,ratind_28,ratind_29,ratind_30
1113065,2296,32,5.53,3.5920,3.486667,3.486667,3.460000,3.586250,3.263333,3.263333,...,2.977368,2.670500,2.781429,2.781429,2.890909,2.917391,2.931250,2.701200,2.509615,2.495926
501732,14967,94,2.04,-1.9900,-2.993333,-1.905000,-2.068000,-1.780000,-1.692857,-2.185000,...,-2.352941,-2.767222,-2.540000,-2.408000,-2.408000,-2.043810,-2.116364,-2.116364,-1.653043,-1.290833
348251,6395,22,-2.86,-1.7000,-2.846000,-2.846000,-2.846000,-2.846000,-1.288333,-1.288333,...,-0.494286,0.088667,0.088667,0.088667,0.235000,0.181176,0.152222,0.042105,0.270500,0.270500
183559,17692,26,-7.28,NaN,-2.330000,2.110000,2.110000,2.110000,2.110000,2.110000,...,4.210000,3.610000,4.133333,4.617143,4.617143,4.617143,4.040000,4.040000,4.040000,4.040000
196180,20977,40,0.73,3.3800,2.717500,3.922000,3.931667,3.931667,3.202857,3.372500,...,2.115294,2.115294,1.873889,2.235263,2.070000,1.879048,1.932727,1.932727,2.030000,2.030000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955751,1504,84,5.39,2.4375,3.076000,3.096667,3.096667,3.111429,2.163750,2.163750,...,0.610000,0.327778,0.343684,0.343684,0.343684,-0.059500,-0.140000,-0.140000,0.000909,0.000909
1267135,2684,65,4.08,1.3700,1.766000,2.070000,2.321429,2.420000,2.420000,2.355556,...,1.927333,1.861250,1.894706,1.808333,1.827895,1.957500,1.984286,1.945000,2.014348,2.014348
84979,16977,64,6.55,6.9875,5.784000,4.738333,4.761429,4.761429,5.180000,4.437778,...,5.021250,4.628824,4.719444,4.854211,4.854211,4.886000,4.886000,4.805714,4.942727,5.016957
768633,10305,61,6.46,7.2300,5.070000,5.070000,5.070000,5.212000,5.516667,4.694286,...,3.930909,3.930909,3.930909,3.595000,3.105385,3.105385,3.431429,3.431429,3.532667,3.532667


In [12]:
rmse = []

for r, n in zip(result, n_neighbors):
    rmse.append(mean_squared_error(valid_df.Rating, [rr if not math.isnan(rr) else 0 for rr in r], squared=False))
    
rmse

[4.958943501881511,
 4.870718617861183,
 4.797151620107247,
 4.7560014576832605,
 4.711012731931596,
 4.687342070434826,
 4.689987192669771,
 4.655813613294245,
 4.63810700094694,
 4.629038319351621,
 4.617699472312557,
 4.605299412491398,
 4.5956531868318296,
 4.594006549216844,
 4.590066561481739,
 4.58956979622174,
 4.583789354508473,
 4.58884042546564,
 4.593179346374154,
 4.592630149034039,
 4.584428977247896,
 4.586313247722498,
 4.5813051562483595,
 4.574532342774899,
 4.56825590332492,
 4.573802306293008]